In [39]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import json
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
from transformers import BertTokenizer, BertModel

# Define label mapping
label_mapping = {
    'Appeal to authority': 0, 'Appeal to fear/prejudice': 1, 'Bandwagon': 2,
    'Black-and-white Fallacy/Dictatorship': 3, 'Causal Oversimplification': 4,
    'Doubt': 5, 'Exaggeration/Minimisation': 6, 'Flag-waving': 7, 
    'Glittering generalities (Virtue)': 8, 'Loaded Language': 9, 
    "Misrepresentation of Someone's Position (Straw Man)": 10, 
    'Name calling/Labeling': 11, 'Obfuscation, Intentional vagueness, Confusion': 12, 
    'Presenting Irrelevant Data (Red Herring)': 13, 'Reductio ad hitlerum': 14, 'Repetition': 15, 
    'Slogans': 16, 'Smears': 17, 'Thought-terminating cliché': 18, 'Whataboutism': 19
}

# Define the MemeDataset class
class MemeDataset(Dataset):
    def __init__(self, annotation_file, tokenizer, label_mapping):
        with open(annotation_file, 'r', encoding='utf-8') as f:
            self.data = json.load(f)
        self.tokenizer = tokenizer
        self.label_mapping = label_mapping

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        text = item['text']
        inputs = self.tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=512)
        input_ids = inputs['input_ids'].squeeze()
        attention_mask = inputs['attention_mask'].squeeze()

        label_indices = [self.label_mapping[label] for label in item['labels']]
        labels = torch.zeros(len(self.label_mapping))
        labels[label_indices] = 1

        return input_ids, attention_mask, labels

# Define the MemeClassifier model
class MemeClassifier(nn.Module):
    def __init__(self, text_model, num_labels):
        super(MemeClassifier, self).__init__()
        self.text_model = text_model
        self.classifier = nn.Linear(self.text_model.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.text_model(input_ids=input_ids, attention_mask=attention_mask)
        text_features = outputs.pooler_output
        logits = self.classifier(text_features)
        return logits

# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = MemeClassifier(BertModel.from_pretrained('bert-base-uncased'), num_labels=len(label_mapping))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
criterion = nn.BCEWithLogitsLoss()

# Training and validation datasets and dataloaders
train_dataset = MemeDataset(
    annotation_file='C:\\Users\\harih\\Downloads\\27\\semeval2024_dev_release\\semeval2024_dev_release\\subtask1\\train_filtered.json',
    tokenizer=tokenizer,
    label_mapping=label_mapping
)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

val_dataset = MemeDataset(
    annotation_file='C:\\Users\\harih\\Downloads\\27\\semeval2024_dev_release\\semeval2024_dev_release\\subtask1\\validation.json',
    tokenizer=tokenizer,
    label_mapping=label_mapping
)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

# Training loop with validation
num_epochs = 15
for epoch in range(num_epochs):
    model.train()
    total_loss, train_correct, train_total = 0, 0, 0
    for input_ids, attention_mask, labels in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}'):
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        optimizer.zero_grad()
        logits = model(input_ids, attention_mask)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        with torch.no_grad():
            predictions = torch.sigmoid(logits).round()
            train_correct += (predictions == labels).all(dim=1).sum().item()
            train_total += labels.size(0)

    train_loss = total_loss / len(train_loader)
    train_accuracy = train_correct / train_total
    print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss}, Train Accuracy: {train_accuracy}")

    # Validation phase
    model.eval()
    val_loss, val_correct, val_total = 0, 0, 0
    all_labels, all_preds = [], []
    with torch.no_grad():
        for input_ids, attention_mask, labels in val_loader:
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
            logits = model(input_ids, attention_mask)
            loss = criterion(logits, labels)
            val_loss += loss.item()
            predictions = torch.sigmoid(logits).round()
            val_correct += (predictions == labels).all(dim=1).sum().item()
            val_total += labels.size(0)
            all_labels.append(labels.cpu())
            all_preds.append(predictions.cpu())
            

    val_loss = val_loss / len(val_loader)
    val_accuracy = val_correct / val_total
    print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

    # Calculate hierarchical F1 score
    all_labels_np = torch.cat(all_labels, dim=0).numpy()
    all_preds_np = torch.cat(all_preds, dim=0).numpy()
    h_f1 = hierarchical_f1_score(all_labels_np, all_preds_np, label_mapping, hierarchy)
    print(f"Validation Hierarchical F1 Score: {h_f1}")


Epoch 1/15: 100%|██████████| 1434/1434 [03:17<00:00,  7.28it/s]


Epoch 1/15, Train Loss: 0.25933822898539705, Train Accuracy: 0.04079497907949791
Validation Loss: 0.2141675880551338, Validation Accuracy: 0.182
Validation Hierarchical F1 Score: 0.9223


Epoch 2/15: 100%|██████████| 1434/1434 [03:22<00:00,  7.09it/s]


Epoch 2/15, Train Loss: 0.21372416453901028, Train Accuracy: 0.15271966527196654
Validation Loss: 0.20962062710523605, Validation Accuracy: 0.18
Validation Hierarchical F1 Score: 0.9199


Epoch 3/15: 100%|██████████| 1434/1434 [03:23<00:00,  7.05it/s]


...

Epoch 13/15, Train Loss: 0.02451835622606056, Train Accuracy: 0.8683751743375174
Validation Loss: 0.3966154611110687, Validation Accuracy: 0.152
Validation Hierarchical F1 Score: 0.9159


Epoch 14/15: 100%|██████████| 1434/1434 [03:22<00:00,  7.07it/s]


Epoch 14/15, Train Loss: 0.023795079214475993, Train Accuracy: 0.8673291492329149
Validation Loss: 0.3985769712328911, Validation Accuracy: 0.148
Validation Hierarchical F1 Score: 0.9122000000000001


Epoch 15/15: 100%|██████████| 1434/1434 [03:22<00:00,  7.09it/s]


Epoch 15/15, Train Loss: 0.0217208201141159, Train Accuracy: 0.8847629009762901
Validation Loss: 0.3931615781188011, Validation Accuracy: 0.158
Validation Hierarchical F1 Score: 0.9136


In [8]:

val_loss, val_correct, val_total = 0, 0, 0
all_labels, all_preds = [], []
with torch.no_grad():
    for input_ids, attention_mask, labels in val_loader:
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        logits = model(input_ids, attention_mask)
        loss = criterion(logits, labels)
        val_loss += loss.item()
        predictions = torch.sigmoid(logits).round()
        val_correct += (predictions == labels).all(dim=1).sum().item()
        val_total += labels.size(0)
        all_labels.append(labels.cpu())
        all_preds.append(predictions.cpu())
            

val_loss = val_loss / len(val_loader)
val_accuracy = val_correct / val_total
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

    # Calculate hierarchical F1 score
all_labels_np = torch.cat(all_labels, dim=0).numpy()
all_preds_np = torch.cat(all_preds, dim=0).numpy()
h_f1 = hierarchical_f1_score(all_labels_np, all_preds_np, label_mapping, hierarchy)
print(f"Validation Hierarchical F1 Score: {h_f1}")


NameError: name 'val_loader' is not defined

In [31]:
hierarchy = {

}

In [33]:
persuasion_hierarchy = {
    'Persuasion': ['Ethos', 'Pathos', 'Logos'],
    'Ethos': ['Ad Hominem', 'Bandwagon', 'Appeal to Authority', 'Glittering Generalities', 'Transfer'],
    'Pathos': ['Appeal to Emotion', 'Exaggeration', 'Loaded Language', 'Flag Waving', 'Appeal to Fear', 'Transfer'],
    'Logos': ['Justification', 'Reasoning', 'Repetition', 'Intentional Vagueness'],
    'Ad Hominem': ['Name Calling', 'Doubt', 'Smears', 'Reductio ad Hitlerum', 'Whataboutism'],
    'Justification': ['Bandwagon', 'Appeal to Authority', 'Flag Waving', 'Appeal to Fear', 'Slogans'],
    'Reasoning': ['Distraction', 'Simplification'],
    'Distraction': ['Straw Man', 'Red Herring', 'Whataboutism'],
    'Simplification': ['Causal Oversimplification', 'Black & White Fallacy', 'Thought Terminating Cliche'],
}

In [35]:
# Function to get all ancestors of a technique in the hierarchy
def get_ancestors(technique, hierarchy):
    ancestors = []
    for parent, children in hierarchy.items():
        if technique in children:
            ancestors.append(parent)
            ancestors.extend(get_ancestors(parent, hierarchy))
    return ancestors

In [37]:
from sklearn.metrics import f1_score

# Assuming your labels are in string format as used in the hierarchy
def hierarchical_f1_score(y_true, y_pred, label_mapping, hierarchy):
    """Calculate the hierarchical F1 score considering the class hierarchy.

    Args:
        y_true (list of list of strings): True labels.
        y_pred (list of list of strings): Predicted labels.
        label_mapping (dict): Mapping from label names to indices.
        hierarchy (dict): Hierarchy of labels.

    Returns:
        float: The hierarchical F1 score.
    """
    # Reverse the label mapping to convert indices to names
    index_to_label = {v: k for k, v in label_mapping.items()}

    # Flatten the true and predicted labels to include their ancestors
    y_true_flat = []
    y_pred_flat = []
    for label_set in y_true:
        for label in label_set:
            label_name = index_to_label[label]
            y_true_flat.extend(get_ancestors(label_name, hierarchy) + [label_name])
    for label_set in y_pred:
        for label in label_set:
            label_name = index_to_label[label]
            y_pred_flat.extend(get_ancestors(label_name, hierarchy) + [label_name])

    # Convert labels names to indices for F1 calculation
    y_true_indices = [label_mapping[label] for label in y_true_flat]
    y_pred_indices = [label_mapping[label] for label in y_pred_flat]

    # Calculate F1 score
    f1 = f1_score(y_true_indices, y_pred_indices, average='micro')
    return f1




In [46]:
def save_model(model, path):
    torch.save(model.state_dict(), path)
    print(f"Model saved to {path}")

In [48]:
model_path = 'C:\\Users\\harih\\Downloads\\27\\bert_meme_model_subtask1.pth'
save_model(model, model_path)

Model saved to C:\Users\harih\Downloads\27\bert_meme_model_subtask1.pth
